In [89]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np

from definitions import ROOT_DIR, main_activity_label_classes
from trainTestValSplit import getTrainTestValSplit
from preProcessing import preProcess
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.metrics import f1_score

In [2]:
train_concat, test_concat, val_concat = getTrainTestValSplit(
    _dataset_path=f'{ROOT_DIR}/dataset/ExtraSensory/ExtraSensory.per_uuid_features_labels'
)

In [54]:
train_x, train_y = preProcess(train_concat)
test_x, test_y = preProcess(test_concat)
val_x, val_y = preProcess(val_concat)

In [55]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)
val_x = val_x.fillna(0)

In [56]:
n_neurons: int = 1024
window_size: int = 1
epochs: int = 20
n_y: int = len(main_activity_label_classes)
batch_size: int = 1024
n_features: int = len(train_x.columns)

In [57]:
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
test_x = test_x.to_numpy()
test_y = test_y.to_numpy()
val_x = val_x.to_numpy()
val_y = val_y.to_numpy()

In [58]:
# Source: https://stackoverflow.com/questions/43114460/is-there-a-way-to-reshape-an-array-that-does-not-maintain-the-original-size-or
def reshape_and_truncate(arr, shape):
    desired_size_factor = np.prod([n for n in shape if n != -1])
    if -1 in shape:  # implicit array size
        desired_size = arr.size // desired_size_factor * desired_size_factor
    else:
        desired_size = desired_size_factor
    return arr.flat[:desired_size].reshape(shape)

In [59]:
def encodeTarget(y):
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    return np_utils.to_categorical(y)

In [60]:
train_y = encodeTarget(train_y)
test_y = encodeTarget(test_y)
val_y = encodeTarget(val_y)

In [61]:
train_x = reshape_and_truncate(train_x, (-1, window_size, n_features))
train_y = reshape_and_truncate(train_y, (-1, window_size, len(main_activity_label_classes)))

test_x = reshape_and_truncate(test_x, (-1, window_size, n_features))
test_y = reshape_and_truncate(test_y, (-1, window_size, len(main_activity_label_classes)))

val_x = reshape_and_truncate(val_x, (-1, window_size, n_features))
val_y = reshape_and_truncate(val_y, (-1, window_size, len(main_activity_label_classes)))

In [63]:
with tf.device('/cpu:0'):
    model = tf.keras.Sequential()

    layer = tf.keras.layers.Normalization(axis=-1)
    layer.adapt(train_x)
    model.add(layer)

    model.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model.add(tf.keras.layers.LSTM(32, return_sequences=True))

    model.add(tf.keras.layers.Dense(n_y, activation=tf.keras.activations.softmax))

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy())#, optimizer=tf.keras.optimizers.SGD(0.00001))
    # model.summary()
    model.fit(x=train_x, y=train_y, validation_data=(val_x, val_y), epochs=epochs, batch_size=batch_size)
    predict = model.predict(test_x)

Epoch 1/20
275/275 [==============================] - 11s 20ms/step - loss: 1.3174 - val_loss: 1.1989
Epoch 2/20
275/275 [==============================] - 4s 16ms/step - loss: 0.8751 - val_loss: 1.0358
Epoch 3/20
275/275 [==============================] - 5s 18ms/step - loss: 0.6535 - val_loss: 1.0318
Epoch 4/20
275/275 [==============================] - 5s 18ms/step - loss: 0.5883 - val_loss: 1.0496
Epoch 5/20
275/275 [==============================] - 5s 17ms/step - loss: 0.5568 - val_loss: 1.0411
Epoch 6/20
275/275 [==============================] - 5s 17ms/step - loss: 0.5356 - val_loss: 1.0430
Epoch 7/20
275/275 [==============================] - 5s 18ms/step - loss: 0.5178 - val_loss: 1.1549
Epoch 8/20
275/275 [==============================] - 5s 18ms/step - loss: 0.5042 - val_loss: 1.0808
Epoch 9/20
275/275 [==============================] - 6s 20ms/step - loss: 0.4929 - val_loss: 1.1142
Epoch 10/20
275/275 [==============================] - 5s 18ms/step - loss: 0.4830 - val_l

In [90]:
_, gold_labels = preProcess(test_concat)
gold_labels += 1

In [91]:
predicted_labels = [np.argmax(pred) for pred in predict]

In [93]:
f1_score(gold_labels, predicted_labels, average='weighted')

0.6892817206080092

In [88]:
metric = tfa.metrics.F1Score(num_classes=len(main_activity_label_classes))
metric.update_state(gold_labels, predicted_labels)
metric.result()

TypeError: in user code:

    File "/Volumes/SEAGATE/code/ML4QS-Assignment3/pip_venv/lib/python3.9/site-packages/tensorflow_addons/metrics/f_scores.py", line 148, in update_state  *
        y_pred = tf.logical_and(y_pred >= threshold, tf.abs(y_pred) > 1e-12)

    TypeError: Cannot convert 1e-12 to EagerTensor of dtype int32


In [81]:
tf.math.confusion_matrix(gold_labels, predicted_labels)

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[15412,   256,   556,   207,   482,     0,     0],
       [    0,  9664,  4939,   318,    34,     0,     1],
       [   29,  1775, 18817,  2794,   389,     0,     7],
       [   15,   494,  3788,  1436,   269,     0,    13],
       [   98,   134,  1537,  1461,  1212,     0,    29],
       [   11,    44,    65,   105,    79,     0,     2],
       [   28,     9,   268,   202,   283,     0,   238]], dtype=int32)>